In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import time
import functools 

In [ ]:
date, bid, ask = np.loadtxt('GBPUSD/GBPUSD1d.txt',
                                unpack=True,
                                delimiter=',',
                                converters={0: lambda x: mdates.datestr2num(x.decode('utf8'))})

In [ ]:
def percentageChange(startPoint, currentPoint):
    try:
        x = ((float(currentPoint)-startPoint)/abs(startPoint))*100.00
        if x == 0.0:
            return 0.00000000001
        else:
            return x
    except:
        return 0.00000000001
        

def patternStorage(data):
    patStartTime = time.time()
    x = len(data)-30
    y = 31 
    while y < x:
        pattern = []
        for i in range(1,31):
            pattern.append(percentageChange(data[y-30], data[y-(30-i)]))
        
        outcomeRange = data[y+20:y+30]
        currentPoint = data[y]
        
        try:
            avgOutcome = functools.reduce(lambda x, y:x+y, outcomeRange)/len(outcomeRange)
        except e:
            print(str(e))
            avgOutcome = 0
            
        futureOutcome = percentageChange(currentPoint,avgOutcome)
        
        patternAr.append(pattern)
        performanceAr.append(futureOutcome)
        
        y += 1
        
    patEndTime = time.time()
    print("Pattern Storage Time:", patEndTime - patStartTime, "seconds")
    
    
def currentPattern():
    for i in range(1,31):
        patForRec.append(percentageChange(avgLine[-31],avgLine[-31+i]))
    
def patternRecognition():
    patFound = False
    plotPattAr = []
    predictedOutcomeAr = []
    
    for eachPattern in patternAr:
        howSim = 0;
        for i in range(len(eachPattern)):
            howSim += (100.00 - abs(percentageChange(eachPattern[i],patForRec[i])))
        
        howSim = howSim/len(eachPattern)
        
        if howSim > 70:
            patdex =   patternAr.index(eachPattern)
            patFound = True
            plotPattAr.append(eachPattern)
    
    predArray = []
    
    
    #xp = [i for i in range(1,31)]
    
    if patFound:
        #fig = plt.figure()
        for eachPatt in plotPattAr:
            futurePoints = patternAr.index(eachPatt)
            
            predictedOutcomeAr.append(performanceAr[futurePoints])
            
            if performanceAr[futurePoints]>patForRec[29]:
                pcolor = "#24bc00"
                predArray.append(1.000)
            else:
                pcolor = "#d40000"  
                predArray.append(-1.000)
                
            '''   
            plt.plot(xp,eachPatt)
            plt.scatter(35, performanceAr[futurePoints],c=pcolor,alpha=.3)
            '''
            
        realOutcomeRange = allData[toWhat+20:toWhat+30]
        realAvgOutcome = functools.reduce(lambda x, y:x+y, realOutcomeRange)/len(realOutcomeRange)
        realMovement = percentageChange(allData[toWhat], realAvgOutcome)
        predictedAvgOutcome = functools.reduce(lambda x, y:x+y, predictedOutcomeAr)/len(predictedOutcomeAr)
        
        predictionAvg = functools.reduce(lambda x, y:x+y, predArray)/len(predArray)
        
        if predictionAvg < 0:
            if realMovement < patForRec[29]:
                accuracyArray.append(100)
            else:
                accuracyArray.append(0)
                
        if predictionAvg > 0:
            if realMovement > patForRec[29]:
                accuracyArray.append(100)
            else:
                accuracyArray.append(0)
        
        '''
        plt.scatter(40, realMovement, c='#54fff7', s=25)
        plt.scatter(40, predictedAvgOutcome, c='#000000', s=25)
       
        plt.plot(xp, patForRec, '#54fff7', linewidth=3)
        plt.grid(True)
        plt.title('Pattern Recognition')
        plt.show()
        '''
            

def graphRawFX():
    fig = plt.figure(figsize=(10,7))
    ax1 = plt.subplot2grid((40,40), (0,0), rowspan=40, colspan=40)
    ax1.plot(date,bid)
    ax1.plot(date,ask)
    plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    for label in ax1.xaxis.get_ticklabels():
        label.set_rotation(45)
    ax1_2 = ax1.twinx()
    ax1_2.fill_between(date,0,(ask-bid), facecolor='g', alpha=.3)
    plt.subplots_adjust(bottom=.23)
    plt.grid(True)
    plt.show()

In [ ]:
dataLength = int(bid.shape[0])
print("Data Length:", dataLength)

toWhat = 2000
allData = ((bid+ask)/2)
trainLen = 2*int(len(allData)/3)

patternAr = [] #differentPatterns
performanceAr = [] #Different Patters performance

patternStorage(allData[:trainLen])

In [ ]:
accuracyArray = []
accuracyLen = 0
samps = 0

while (trainLen+toWhat) < dataLength:
    
    avgLine = allData[trainLen:trainLen+toWhat]
    patForRec = [] #currentpattern
    
    currentPattern()
    patternRecognition()
    
    if len(accuracyArray) > accuracyLen:
        accuracy = functools.reduce(lambda x, y:x+y, accuracyArray)/len(accuracyArray)
        print("accuracy after ", samps , ' is:', accuracy)
        accuracyLen = len(accuracyArray)
    
    #moveOn = input('press Enter to continue...')
    samps += 1
    toWhat += 1